In [37]:
# == importar bibliotecas == 

from bs4 import BeautifulSoup as bs
import unicodedata as ucd
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from itertools import chain 

In [40]:
# == settando variáveis == 

JORNAL = 'PL'
LABEL = 'direita' 
URL = "https://partidoliberal.org.br/noticias-partido-liberal/"
NUM_PAG = 568
class_1 = 'col-md-8'
class_2 = 'post__content'

In [10]:
# == obter html == 

def get_soup(url):
    response = requests.get(url)
    html = response.content
    soup = bs(html, 'lxml')
    return soup

In [12]:
# == obter dataframe do texto == 

def return_df(url, text):
    return pd.DataFrame({
        'jornal': JORNAL,
        'url': url,
        'artigo': text,
        'label': LABEL
    }, index=[1])

In [26]:
# == lista de URLs ==

list_url = []

In [27]:
# == coletar os sites de artigos == 

def get_links(pag_num):
    cur_list = []
    soup = get_soup(f'{URL}page/{pag_num}')
    for noticia in soup.find_all("div", class_=class_1):
        for artigo in noticia.find_all("a", href=True):
            cur_list.append(artigo.get('href'))
    return cur_list

for n in range(NUM_PAG):
    list_url.append(get_links(n))



  0%|          | 0/568 [00:00<?, ?it/s]

  2%|▏         | 12/568 [00:00<00:04, 119.50it/s]

  4%|▍         | 24/568 [00:02<00:37, 14.64it/s] 

  6%|▋         | 36/568 [00:04<00:55,  9.51it/s]

  8%|▊         | 48/568 [00:07<01:10,  7.34it/s]

 11%|█         | 60/568 [00:10<01:31,  5.54it/s]

 13%|█▎        | 72/568 [00:13<01:32,  5.35it/s]

 15%|█▍        | 84/568 [00:15<01:32,  5.25it/s]

 17%|█▋        | 96/568 [00:19<01:43,  4.57it/s]

 19%|█▉        | 108/568 [00:21<01:37,  4.72it/s]

 21%|██        | 120/568 [00:23<01:26,  5.15it/s]

 23%|██▎       | 132/568 [00:25<01:23,  5.23it/s]

 25%|██▌       | 144/568 [00:27<01:18,  5.38it/s]

 27%|██▋       | 156/568 [00:30<01:21,  5.08it/s]

 30%|██▉       | 168/568 [00:32<01:20,  4.95it/s]

 32%|███▏      | 180/568 [00:35<01:19,  4.90it/s]

 34%|███▍      | 192/568 [00:37<01:10,  5.34it/s]

 36%|███▌      | 204/568 [00:39<01:11,  5.11it/s]

 38%|███▊      | 216/568 [00:42<01:15,  4.68it/s]

 40%|████      | 228/568 [00:45<01:13,  4.64i

In [36]:
# == remover url duplicado == 

list_url = list(chain.from_iterable(list_url))
list_url = list(set(list_url))

In [61]:
# == remover links que não são notícias == 

list_url = [
    url for url in list_url if 'https://partidoliberal.org.br/noticias-partido-liberal/page/' not in url
]

3402

In [ ]:
# == obter conteúdo da notícia == 

def get_artigo(url):
    soup = get_soup(url)
    list_text = []
    for div in soup.find_all("div", class_=class_2):
        for p in div.find_all("p"):
            list_text.append(ucd.normalize("NFKD", p.text))
        return return_df(url, " ".join(list_text))

res = []
for url in list_url:
    res.append(get_artigo(url))
scrap = pd.concat(res)

In [75]:
# == salvar o scrapping == 

scrap.to_parquet('../dataset/scrapping/PL.parquet')

In [76]:
scrap.head(10)

,jornal,url,artigo,label
1,PL,https://partidoliberal.org.br/aprovado-projeto...,Rio Grande do Norte – A Assembleia Legislativa...,direita
1,PL,https://partidoliberal.org.br/deputado-federal...,"O Projeto prevê reclusão de 3 a 6 anos, alé...",direita
1,PL,https://partidoliberal.org.br/medeiros-lamenta...,Brasília – O deputado federal José Medeiros ...,direita
1,PL,https://partidoliberal.org.br/o-presidente-bol...,"Brasília – O líder do PL na Câmara, deputad...",direita
1,PL,https://partidoliberal.org.br/senador-wellingt...,Brasília – A duplicação da BR-364 entre Ron...,direita
1,PL,https://partidoliberal.org.br/projeto-de-carlo...,Brasília – Aguarda apreciação em Plenário ...,direita
1,PL,https://partidoliberal.org.br/comissao-aprova-...,BrasÃ­lia – A ComissÃ£o de SeguranÃ§a PÃob...,direita
1,PL,https://partidoliberal.org.br/jair-bolsonaro-s...,Brasília – O dia 11 de agosto tornou-se a dat...,direita
1,PL,https://partidoliberal.org.br/michelle-bolsona...,"A cerimônia foi realizada neste sábado (6), ...",direita
1,PL,https://partidoliberal.org.br/bolsonaro-destac...,Brasília – O presidente Jair Bolsonaro (PL) p...,direita
